#Common functions for all models

### Installing Packages & Importing Libraries

In [ ]:
!pip install pyreadstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 29.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import transformers

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from transformers import AutoTokenizer,AutoModelForCausalLM
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Functions for WVS Dataset

In [ ]:
#Loading the WVS file and storing it in a dataframe format

def get_wvs_df():
    wvs_df = pd.read_csv('/content/drive/MyDrive/MSc Applied Data Science/ADS thesis Mijntje/WVS_Moral.csv') #WVS_Moral is a subset of the full data for just the moral questions
    wvs_df_country_names = pd.read_csv('/content/drive/MyDrive/MSc Applied Data Science/ADS thesis Mijntje/Country_Codes_Names-correct.csv')
    wvs_df = wvs_df.set_index('B_COUNTRY').join(wvs_df_country_names.set_index('B_COUNTRY'), how='left')
    return wvs_df

In [ ]:
#Listing all the countries we are gonna use in our study from the WVS dataset

COUNTRIES_WVS_W7_ALL = [
    'Andorra', 'Argentina', 'Armenia', 'Australia', 'Bangladesh', 'Bolivia', 'Brazil', 'Canada',
    'Chile', 'China', 'Colombia', 'Cyprus', 'Ecuador', 'Egypt', 'Ethiopia', 'Germany', 'Greece',
    'Guatemala', 'Indonesia', 'Iran', 'Iraq', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya',
    'Kyrgyzstan', 'Lebanon', 'Libya', 'Malaysia', 'Maldives', 'Mexico', 'Mongolia', 'Morocco',
    'Myanmar', 'Netherlands', 'New Zealand', 'Nicaragua', 'Nigeria', 'Pakistan', 'Peru',
    'Philippines', 'Romania', 'Russia', 'Singapore', 'South Korea', 'Taiwan ROC', 'Tajikistan',
    'Thailand', 'Tunisia', 'Turkey', 'Ukraine', 'United States', 'Venezuela',
    'Vietnam', 'Zimbabwe'
]

# COUNTRIES_WVS_W7_ALL = [
#     'Andorra', 'Argentina', 'Armenia', 'Australia', 'Bangladesh', 'Bolivia', 'Cyprus', 'Ethiopia', 'Germany', 'Greece',
#     'Guatemala', 'Indonesia', 'Iran', 'Iraq', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya',
#     'Kyrgyzstan', 'Lebanon', 'Libya', 'Malaysia', 'Maldives', 'Mongolia', 'Morocco',
#     'Myanmar', 'Netherlands', 'New Zealand', 'Nicaragua', 'Nigeria', 'Pakistan', 'Peru',
#     'Philippines', 'Romania', 'Russia', 'Singapore', 'South Korea', 'Taiwan ROC', 'Tajikistan',
#     'Thailand', 'Tunisia', 'Turkey', 'Ukraine', 'United States', 'Venezuela',
#     'Vietnam', 'Zimbabwe'
# ]

In [ ]:
#Creating the questions for the WVS dataset

W7_QUESTIONS = ['Q'+str(i) for i in range(177, 196)]

W7_QUESTIONS_TEXT = ['claiming government benefits to which you are not entitled',
                     'avoiding a fare on public transport',
                     'stealing property',
                     'cheating on taxes',
                     'someone accepting a bribe in the course of their duties',
                     'homosexuality',
                     'prostitution',
                     'abortion',
                     'divorce',
                     'sex before marriage',
                     'suicide',
                     'euthanasia',
                     'for a man to beat his wife',
                     'parents beating children',
                     'violence against other people',
                     'terrorism as a political, ideological or religious mean',
                     'having casual sex',
                     'political violence',
                     'death penalty']

QUESTION_WAVES_WVS = {7: (W7_QUESTIONS, W7_QUESTIONS_TEXT)}

In [ ]:
#Scaling the survey results between -1 and 1 and calculating the mean rating for a every survey question and culture

MINUS = 5.5
DIVIDE = 4.5

def get_wvs_ratings(wvs_df, culture, q):
    df = wvs_df[['Country_Names', q]]
    df = df.loc[df['Country_Names'] == culture]
    if len(df) == 0:
        return None

    ratings = df.loc[df[q] > 0][q]
    if ratings.empty:
        ratings = 0
    else:
      ratings = ((ratings - MINUS) / DIVIDE).mean()
    return ratings

In [ ]:
#Generating the WVS prompts

def wvs_gpt2_prompts_ratings_multiple_tokens(wvs_df:pd.DataFrame, culture ='', prompt_mode ='in', wave = 7, ):
    prompts = {}
    culture_prompt = ''
    if len(culture) > 0:
        if prompt_mode == 'in':
            print("The token used is IN")
            culture_prompt = f'In {culture} '
        elif prompt_mode == 'people':
            print("The token used is PEOPLE")
            culture_prompt = f'People in {culture} believe '

    questions, questions_text = QUESTION_WAVES_WVS[wave]

    for q,q_text in zip(questions, questions_text):
        rating_answers = get_wvs_ratings(wvs_df, culture, q) #getting the original ratings

        prompts[q_text] = []
        for (prompt_head_moral, prompt_head_nonmoral) in TOKEN_PAIRS:
            prompt_moral = f'{culture_prompt}{q_text} is {prompt_head_moral}.'
            prompt_nonmoral = f'{culture_prompt}{q_text} is {prompt_head_nonmoral}.'

            prompts[q_text].append((prompt_moral, prompt_nonmoral,rating_answers))

    return prompts

### Functions for PEW Dataset

In [ ]:
#Loading the PEW file, pre-processing it and storing it in a dataframe

def get_pew_df():
    pew_data_original = pd.read_spss('/content/drive/MyDrive/MSc Applied Data Science/ADS thesis Mijntje/Pew Research Global Attitudes Project Spring 2013 Dataset for web.sav')

    filtered_columns = pew_data_original.filter(regex='^Q84[A-H]|COUNTRY').copy()

    filtered_columns.rename(columns={'COUNTRY': 'Country_Names'}, inplace=True)

    replace_map = {
        'Morally acceptable': 1,
        'Not a moral issue': 0,
        'Morally unacceptable': -1,
        'Depends on situation (Volunteered)': 0,
        'Refused': 0,
        "Don't know": 0
    }

    filtered_columns.replace(replace_map, inplace=True)

    for col in filtered_columns.columns[1:]:
        filtered_columns[col] = pd.to_numeric(filtered_columns[col], errors='coerce')

    return filtered_columns

In [ ]:
#Listing all the countries we are gonna use in our study from the PEW dataset

COUNTRIES_PEW_ALL = [
    'United States', 'Czech Republic', 'South Korea', 'Canada', 'France', 'Germany',
    'Spain', 'Mexico', 'Chile', 'Australia', 'Russia', 'Britain', 'Turkey', 'Greece',
    'Egypt', 'Poland', 'Senegal', 'Italy', 'Brazil', 'Lebanon', 'Nigeria', 'Japan',
    'Malaysia', 'Kenya', 'Indonesia', 'Uganda', 'Jordan', 'Argentina', 'Philippines',
    'Tunisia', 'China', 'Pakistan', 'Ghana', 'South Africa', 'Palestinian territories',
    'Israel', 'Bolivia', 'Venezuela', 'El Salvador'
]

In [ ]:
#Creating the questions for the PEW dataset

PEW_QUESTIONS = ['Q84' + chr(i) for i in range(ord('A'), ord('H')+1)]

PEW_QUESTIONS_TEXT = ['using contraceptives',
                      'getting a divorce',
                      'having an abortion',
                      'homosexuality',
                      'drinking alcohol',
                      'married people having an affair',
                      'gambling',
                      'sex between unmarried adults']

QUESTION_WAVES_PEW = {13: (PEW_QUESTIONS, PEW_QUESTIONS_TEXT)}

In [ ]:
#Calculating the mean rating for a every question and culture

def get_pew_ratings(pew_df, culture, q):
    df = pew_df[['Country_Names', q]]
    df = df.loc[df['Country_Names'] == culture]
    if df.empty:
        print("No data found for culture:", culture)
        return None

    mean_rating = df[q].mean()

    if pd.isna(mean_rating):
        print("Problem: Mean calculation resulted in NaN for culture:", culture)
        return None

    return mean_rating

In [ ]:
#Generating the PEW prompts

def pew_gpt2_prompts_ratings_multiple_tokens(pew_df:pd.DataFrame, culture ='', prompt_mode ='in', wave = 13, ):
    prompts = {}
    culture_prompt = ''
    if len(culture) > 0:
        if prompt_mode == 'in':
            culture_prompt = f'In {culture} '
            print("Inside IN")
        elif prompt_mode == 'people':
            culture_prompt = f'People in {culture} believe '
            print("Inside PEOPLE")

    questions, questions_text = QUESTION_WAVES_PEW[wave]

    for q,q_text in zip(questions, questions_text):
        rating_answers = get_pew_ratings(pew_df, culture, q) #getting the original ratings

        prompts[q_text] = []
        for (prompt_head_moral, prompt_head_nonmoral) in TOKEN_PAIRS:
            prompt_moral = f'{culture_prompt}{q_text} is {prompt_head_moral}.'
            prompt_nonmoral = f'{culture_prompt}{q_text} is {prompt_head_nonmoral}.'

            prompts[q_text].append((prompt_moral, prompt_nonmoral,rating_answers))

    return prompts

## Other functions

### normalize_log_prob_diffs() and calculate_correlation()

In [ ]:
#Normalizing the log probability differences in a scale between -1 and 1

def normalize_log_prob_diffs(log_prob_diffs):
    min_log_prob = np.min(log_prob_diffs)
    max_log_prob = np.max(log_prob_diffs)
    normalized_log_probs = 2 * (log_prob_diffs - min_log_prob) / (max_log_prob - min_log_prob) - 1
    return normalized_log_probs

#Calculating the Pearson correlation between model and survey scores

def calculate_correlation(survey_scores, log_prob_diffs):
  if len(survey_scores) != len(log_prob_diffs):
      print(f"Error: Mismatched lengths. Survey scores length: {len(survey_scores)}, Log prob diffs length: {len(log_prob_diffs)}")
      return None, None, None

  normalized_log_probs = normalize_log_prob_diffs(log_prob_diffs)
  try:
        correlation, p_value = pearsonr(survey_scores, normalized_log_probs)
  except Exception as e:
        print("Error during correlation calculation:", e)
        return None, None, None
  return correlation, normalized_log_probs, p_value

### get_batch_last_token_log_prob() function

In [ ]:
#Calculating the log probabilities of each of the 2 tokens

def get_batch_last_token_log_prob(lines, model, tokenizer, use_cuda=False, end_with_period=True):
    eos_token = tokenizer.eos_token or tokenizer.sep_token
    if eos_token is None:
        raise ValueError("Neither eos_token nor sep_token is set in the tokenizer.")

    lines = [line + eos_token for line in lines]  #Appending EOS to the end of each line

    tokenizer.pad_token = tokenizer.eos_token

    tok_moral = tokenizer.batch_encode_plus(lines, return_tensors='pt', padding='longest', add_special_tokens=True)
    input_ids = tok_moral['input_ids']
    attention_mask = tok_moral['attention_mask']
    lines_len = torch.sum(attention_mask, dim=1)

    remove_from_end = 2 if end_with_period else 1  #If there is a period remove it as well as the EOS token else remove only the EOS token
    tokens_wanted = lines_len - remove_from_end

    if use_cuda:
        device = next(model.parameters()).device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        torch.cuda.empty_cache()

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids, return_dict=True)
        logits = outputs.logits
        if use_cuda:
            logits = logits.detach().cpu()

    logits_probs = F.log_softmax(logits, dim=-1)

    batch_indices = torch.arange(input_ids.size(0))
    token_indices = tokens_wanted - 1   #take the index of the token we need. index of 1st element is 0 so we have to do -1
    next_token_indices = input_ids[batch_indices, tokens_wanted].cpu()

    log_probs = logits_probs[batch_indices, token_indices, next_token_indices]

    # for line in lines:
    #     print(f"Processed line: {line}")

    return log_probs

### get_log_prob_difference() function

In [ ]:
#Calculating the average log probability differences between moral and non-moral prompts

def get_log_prob_difference(pairs, moral_index, nonmoral_index, model, tokenizer, use_cuda):
    question_average_lm_score = []
    average_moral_score = []
    average_nonmoral_score = []

    all_prompts = []
    for rating in pairs:
        moral_prompt = rating[moral_index]
        nonmoral_prompt = rating[nonmoral_index]
        all_prompts.append(moral_prompt)
        all_prompts.append(nonmoral_prompt)

    logprobs = get_batch_last_token_log_prob(all_prompts, model, tokenizer, use_cuda)

    for i in range(0, len(logprobs), 2):
        moral_logprob = logprobs[i]
        nonmoral_logprob = logprobs[i + 1]
        lm_score = moral_logprob - nonmoral_logprob

        question_average_lm_score.append(lm_score)
        average_moral_score.append(moral_logprob)
        average_nonmoral_score.append(nonmoral_logprob)

    lm_score = np.mean(question_average_lm_score)
    moral_score = np.mean(average_moral_score)
    nonmoral_score = np.mean(average_nonmoral_score)

    return lm_score, moral_score, nonmoral_score

### compare_token_pair() functions for both datasets

In [ ]:
def compare_wvs_token_pairs(model_name, cultures=None, wave=7, excluding_topics=[],
                            excluding_cultures=[], model=None, tokenizer=None, use_cuda=False, prompt_mode='in'):

    if model is None or tokenizer is None:
        tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
        model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
        if use_cuda:
            model = model.cuda()

    wvs_df = get_wvs_df()

    results_all = []

    for culture in tqdm(cultures):
        if culture in excluding_cultures:
            continue
        prompts = wvs_gpt2_prompts_ratings_multiple_tokens(wvs_df, culture, prompt_mode)

        culture_name = culture if culture else 'universal'
        for question, rating_pairs in prompts.items():
            if any(excluded_topic in question for excluded_topic in excluding_topics):
                continue

            lm_score, moral_log_probs, nonmoral_log_probs = get_log_prob_difference(rating_pairs, 0, 1, model, tokenizer, use_cuda)

            if isinstance(moral_log_probs, torch.Tensor):
              moral_log_probs = moral_log_probs.item()
            if isinstance(nonmoral_log_probs, torch.Tensor):
              nonmoral_log_probs = nonmoral_log_probs.item()
            if isinstance(lm_score, torch.Tensor):
              lm_score = lm_score.item()

            wvs_score = rating_pairs[0][2]  #All token pairs have the same wvs_score

            row = {
                'country': culture_name, 'topic': question, 'wvs_score': wvs_score,
                'moral log prob': moral_log_probs, 'non moral log probs': nonmoral_log_probs,
                'log prob difference': lm_score
            }

            results_all.append(row)

    df = pd.DataFrame(results_all)

    survey_scores = df['wvs_score'].values
    log_prob_diffs = df['log prob difference'].values

    correlation, normalized_log_probs, p_value = calculate_correlation(survey_scores, log_prob_diffs)

    print(normalized_log_probs)

    df['normalized log prob difference'] = normalized_log_probs

    metrics = {
        'Pearson correlation coefficient': correlation,
        'P value': p_value
    }

    return df, metrics

In [ ]:
def compare_pew_token_pairs(model_name, cultures=None, wave=7, excluding_topics=[],
                            excluding_cultures=[], model=None, tokenizer=None, use_cuda=False, prompt_mode='in'):

    if model is None or tokenizer is None:
        tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
        model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
        if use_cuda:
            model = model.cuda()

    pew_df = get_pew_df()

    results_all = []

    for culture in tqdm(cultures):
        if culture in excluding_cultures:
            continue
        prompts = pew_gpt2_prompts_ratings_multiple_tokens(pew_df, culture, prompt_mode)

        culture_name = culture if culture else 'universal'
        for question, rating_pairs in prompts.items():
            if any(excluded_topic in question for excluded_topic in excluding_topics):
                continue

            lm_score, moral_log_probs, nonmoral_log_probs = get_log_prob_difference(rating_pairs, 0, 1, model, tokenizer, use_cuda)

            if isinstance(moral_log_probs, torch.Tensor):
              moral_log_probs = moral_log_probs.item()
            if isinstance(nonmoral_log_probs, torch.Tensor):
              nonmoral_log_probs = nonmoral_log_probs.item()
            if isinstance(lm_score, torch.Tensor):
              lm_score = lm_score.item()

            pew_score = rating_pairs[0][2]  #All token pairs have the same pew_score

            row = {
                'country': culture_name, 'topic': question, 'pew_score': pew_score,
                'moral log prob': moral_log_probs, 'non moral log probs': nonmoral_log_probs,
                'log prob difference': lm_score
            }

            results_all.append(row)

    df = pd.DataFrame(results_all)

    survey_scores = df['pew_score'].values
    log_prob_diffs = df['log prob difference'].values

    correlation, normalized_log_probs, p_value = calculate_correlation(survey_scores, log_prob_diffs)

    df['normalized log prob difference'] = normalized_log_probs

    metrics = {
        'Pearson correlation coefficient': correlation,
        'P value': p_value
    }

    return df, metrics

## Functions only for GPT2 models

In [ ]:
#Loading the GPT2 model and the tokenizer

def get_gpt2_model(modelname = 'gpt2', use_cuda = True, device = 'cuda:0'):  #'gpt2','gpt2-medium','gpt2-large','gpt2-xl'
    model = GPT2LMHeadModel.from_pretrained(modelname)
    tokenizer = GPT2Tokenizer.from_pretrained(modelname)

    if use_cuda and torch.cuda.is_available():
        torch.cuda.empty_cache()
        device = torch.device(device)
        model.cuda(device)

    return tokenizer, model

In [ ]:
def compare_wvs_token_pairs_gpt2(model_name, cultures=None, wave=7, excluding_topics=[],
                                 excluding_cultures=[], model=None, tokenizer=None, use_cuda=False, prompt_mode='in'):

    if model is None or tokenizer is None:
        tokenizer, model = get_gpt2_model(model_name, use_cuda)

    wvs_df = get_wvs_df()

    gpt2_all = []

    for culture in tqdm(cultures):
        if culture in excluding_cultures:
            continue
        prompts = wvs_gpt2_prompts_ratings_multiple_tokens(wvs_df, culture, prompt_mode)

        culture_name = culture if culture else 'universal'
        for question, rating_pairs in prompts.items():
            if any(excluded_topic in question for excluded_topic in excluding_topics):
                continue

            lm_score, moral_log_probs, nonmoral_log_probs = get_log_prob_difference(rating_pairs, 0, 1, model, tokenizer, use_cuda)

            wvs_score = rating_pairs[0][2]  #All token pairs have the same wvs_score

            row = {
                'country': culture_name, 'topic': question, 'wvs_score': wvs_score,
                'moral log prob': moral_log_probs, 'non moral log probs': nonmoral_log_probs,
                'log prob difference': lm_score
            }

            gpt2_all.append(row)

    df = pd.DataFrame(gpt2_all)

    survey_scores = df['wvs_score'].values
    log_prob_diffs = df['log prob difference'].values

    correlation, normalized_log_probs, p_value = calculate_correlation(survey_scores, log_prob_diffs)
    if correlation is not None:
      print("Correlation calculated successfully")
    else:
      print("Failed to calculate correlation due to data length mismatch")

    df['normalized log prob difference'] = normalized_log_probs

    metrics = {
        'Pearson correlation coefficient': correlation,
        'P value': p_value
    }

    return df, metrics

In [ ]:
def compare_pew_token_pairs_gpt2(cultures=None, wave=7, model_name='gpt2',
                                 excluding_topics=[], excluding_cultures=[], model=None, tokenizer=None,
                                 use_cuda=False, prompt_mode='in'):
    if model is None or tokenizer is None:
        tokenizer, model = get_gpt2_model(model_name, use_cuda)

    pew_df = get_pew_df()

    gpt2_all = []

    for culture in tqdm(cultures):
        if culture in excluding_cultures:
            continue
        prompts = pew_gpt2_prompts_ratings_multiple_tokens(pew_df, culture, prompt_mode)

        culture_name = culture if culture else 'universal'
        for question, rating_pairs in prompts.items():
            if any(excluded_topic in question for excluded_topic in excluding_topics):
                continue

            lm_score, moral_log_probs, nonmoral_log_probs = get_log_prob_difference(rating_pairs, 0, 1, model, tokenizer, use_cuda)

            pew_score = rating_pairs[0][2]  #All token pairs have the same pew_score

            row = {
                'country': culture_name, 'topic': question, 'pew_score': pew_score,
                'moral log prob': moral_log_probs, 'non moral log probs': nonmoral_log_probs,
                'log prob difference': lm_score
            }

            gpt2_all.append(row)

    df = pd.DataFrame(gpt2_all)

    survey_scores = df['pew_score'].values
    log_prob_diffs = df['log prob difference'].values

    correlation, normalized_log_probs, p_value = calculate_correlation(survey_scores, log_prob_diffs)
    if correlation is not None:
      print("Correlation calculated successfully")
    else:
      print("Failed to calculate correlation due to data length mismatch")

    df['normalized log prob difference'] = normalized_log_probs

    metrics = {
        'Pearson correlation coefficient': correlation,
        'P value': p_value
    }

    return df, metrics

# Function to create combined df with averaged scores over prompt styles

In [ ]:
def generate_combined_df(df_people, df_in, dataset):

  # now create the combined df

  #Step 1: Concatenate the DataFrames column-wise
  combined_df = pd.concat([df_people, df_in], axis=1)

  # Assuming the 'log prob difference' columns have the same name, differentiate them
  combined_df.columns = [f'{col}_people' if i < len(df_people.columns) else f'{col}_in' for i, col in enumerate(combined_df.columns)]

  # Step 2: Calculate the average log probability difference
  combined_df['average log prob difference'] = (combined_df['log prob difference_people'] + combined_df['log prob difference_in']) / 2

  combined_df['average normalized log prob'] = (combined_df['normalized log prob difference_people'] + combined_df['normalized log prob difference_in']) / 2

  if dataset == 'wvs':

    # Step 3: Remove the 'country_in', 'topic_in', and 'wvs_score_in' columns
    columns_to_remove = ['country_in', 'topic_in', 'wvs_score_in']
    combined_df.drop(columns=columns_to_remove, inplace=True)

    # Step 4: Rename the columns
    combined_df.rename(columns={
      'country_people': 'country',
      'topic_people': 'topic',
      'wvs_score_people': 'wvs_score'
    }, inplace=True)

  elif dataset == 'pew':

    # Step 3: Remove the 'country_in', 'topic_in', and 'wvs_score_in' columns
    columns_to_remove = ['country_in', 'topic_in', 'pew_score_in']
    combined_df.drop(columns=columns_to_remove, inplace=True)

    # Step 4: Rename the columns
    combined_df.rename(columns={
      'country_people': 'country',
      'topic_people': 'topic',
      'pew_score_people': 'pew_score'
    }, inplace=True)


  # Print the resulting DataFrame
  display(combined_df)
  return(combined_df)


In [ ]:
def generate_scores_other(model, cultures=COUNTRIES_WVS_W7_ALL):

  # WVS scores

  if model == 'QWEN':
    model_name = "Qwen/Qwen2-0.5B"

  elif model == 'OPT-125':
    model_name = "facebook/opt-125m"

  elif model == 'BLOOM':
    model_name = "bigscience/bloomz-560m"

  # first 'in' prompt type
  df_model_WVS_in, _ = compare_wvs_token_pairs(model_name=model_name, cultures=COUNTRIES_WVS_W7_ALL, use_cuda=False, prompt_mode='in')
  df_model_WVS_people, _ = compare_wvs_token_pairs(model_name=model_name, cultures=COUNTRIES_WVS_W7_ALL, use_cuda=False, prompt_mode='people')

  save_dir_in = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis Mijntje/Mijn data/wvs_w7_{model}_token_pairs_in-MIJNTJE-FINAL.csv'
  df_model_WVS_in.to_csv(save_dir_in, index=False)

  save_dir_people = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis Mijntje/Mijn data/wvs_w7_{model}_token_pairs_people-MIJNTJE-FINAL.csv'
  df_model_WVS_people.to_csv(save_dir_people, index=False)

  combined_df = generate_combined_df(df_model_WVS_people, df_model_WVS_in, 'wvs')
  save_dir = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis Mijntje/Mijn data/wvs_w7_{model}_token_pairs_BOTH-MIJNTJE-FINAL.csv'
  combined_df.to_csv(save_dir, index=False)

  # PEW scores
  df_model_PEW_in, _ = compare_pew_token_pairs(model_name=model_name, cultures=COUNTRIES_PEW_ALL, use_cuda=False, prompt_mode='in')
  df_model_PEW_people, _ = compare_pew_token_pairs(model_name=model_name, cultures=COUNTRIES_PEW_ALL, use_cuda=False, prompt_mode='people')

  save_dir_in = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis Mijntje/Mijn data/PEW_{model}_token_pairs_in-MIJNTJE-FINAL.csv'
  df_model_PEW_in.to_csv(save_dir_in, index=False)

  save_dir_people = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis Mijntje/Mijn data/PEW_{model}_token_pairs_people-MIJNTJE-FINAL.csv'
  df_model_PEW_people.to_csv(save_dir_people, index=False)

  combined_df = generate_combined_df(df_model_PEW_people, df_model_PEW_in, 'pew')
  save_dir = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis Mijntje/Mijn data/PEW_{model}_token_pairs_BOTH-MIJNTJE-FINAL.csv'
  combined_df.to_csv(save_dir, index=False)

  return combined_df

In [ ]:
def generate_scores_gpt(model, cultures=COUNTRIES_WVS_W7_ALL):

  # WVS scores

  if model == 'gpt2':
    model_name = "gpt2"

  elif model == 'gpt2-medium':
    model_name = "gpt2-medium"

  elif model == 'gpt2-large':
    model_name = "gpt2-large"


  # first 'in' prompt type
  df_model_WVS_in, _ = compare_wvs_token_pairs_gpt2(model_name=model_name, cultures=COUNTRIES_WVS_W7_ALL, use_cuda=False, prompt_mode='in')
  df_model_WVS_people, _ = compare_wvs_token_pairs_gpt2(model_name=model_name, cultures=COUNTRIES_WVS_W7_ALL, use_cuda=False, prompt_mode='people')

  save_dir_in = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis Mijntje/Mijn data/wvs_w7_{model}_token_pairs_in-MIJNTJE-FINAL.csv'
  df_model_WVS_in.to_csv(save_dir_in, index=False)

  save_dir_people = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis Mijntje/Mijn data/wvs_w7_{model}_token_pairs_people-MIJNTJE-FINAL.csv'
  df_model_WVS_people.to_csv(save_dir_people, index=False)

  combined_df = generate_combined_df(df_model_WVS_people, df_model_WVS_in, 'wvs')
  save_dir = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis Mijntje/Mijn data/wvs_w7_{model}_token_pairs_BOTH-MIJNTJE-FINAL.csv'
  combined_df.to_csv(save_dir, index=False)

  # PEW scores
  df_model_PEW_in, _ = compare_pew_token_pairs_gpt2(model_name=model_name, cultures=COUNTRIES_PEW_ALL, use_cuda=False, prompt_mode='in')
  df_model_PEW_people, _ = compare_pew_token_pairs_gpt2(model_name=model_name, cultures=COUNTRIES_PEW_ALL, use_cuda=False, prompt_mode='people')

  save_dir_in = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis Mijntje/Mijn data/PEW_{model}_token_pairs_in-MIJNTJE-FINAL.csv'
  df_model_PEW_in.to_csv(save_dir_in, index=False)

  save_dir_people = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis Mijntje/Mijn data/PEW_{model}_token_pairs_people-MIJNTJE-FINAL.csv'
  df_model_PEW_people.to_csv(save_dir_people, index=False)

  combined_df = generate_combined_df(df_model_PEW_people, df_model_PEW_in, 'pew')
  save_dir = f'/content/drive/MyDrive/MSc Applied Data Science/ADS thesis Mijntje/Mijn data/PEW_{model}_token_pairs_BOTH-MIJNTJE-FINAL.csv'
  combined_df.to_csv(save_dir, index=False)

  return combined_df


In [ ]:
TOKEN_PAIRS = [ ('always justifiable', 'never justifiable'),
                ('right','wrong'),
                ('morally good','morally bad'),
                ('ethically right', 'ethically wrong'),
                ('ethical','unethical')]

# TOKEN_PAIRS = [ ('always justifiable', 'never justifiable')]

In [ ]:
# GPT2_scores = generate_scores_gpt('gpt2')
GPT2MEDIUM_scores = generate_scores_gpt('gpt2-medium')
GPT2LARGE_scores = generate_scores_gpt('gpt2-large')

QWEN_scores = generate_scores_other('QWEN')
OPT_scores = generate_scores_other('OPT-125')
BLOOM_scores = generate_scores_other('BLOOM')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

Inside PEOPLE


 97%|█████████▋| 38/39 [02:54<00:04,  4.64s/it]

Inside PEOPLE


100%|██████████| 39/39 [02:59<00:00,  4.60s/it]


,country,topic,pew_score,moral log prob_people,non moral log probs_people,log prob difference_people,normalized log prob difference_people,moral log prob_in,non moral log probs_in,log prob difference_in,normalized log prob difference_in,average log prob difference,average normalized log prob
0,United States,using contraceptives,0.459082,-1.773682,-1.741267,-0.032415,0.057553,-3.235333,-2.892409,-0.342924,0.077979,-0.187669,0.067766
1,United States,getting a divorce,0.132735,-1.497131,-1.414646,-0.082485,-0.144656,-1.728799,-1.593457,-0.135342,0.354266,-0.108914,0.104805
2,United States,having an abortion,-0.296407,-1.432965,-1.421814,-0.011150,0.143430,-2.149518,-2.177929,0.028411,0.572217,0.008630,0.357824
3,United States,homosexuality,-0.158683,-1.547088,-1.713072,0.165984,0.858788,-1.993815,-2.200828,0.207012,0.809933,0.186498,0.834360
4,United States,drinking alcohol,0.177645,-1.817726,-1.694824,-0.122901,-0.307875,-2.058182,-1.973608,-0.084575,0.421836,-0.103738,0.056980
...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,El Salvador,homosexuality,-0.535354,-1.304779,-1.467671,0.162892,0.846301,-2.131861,-2.481676,0.349815,1.000000,0.256354,0.923150
308,El Salvador,drinking alcohol,-0.714646,-1.597174,-1.495350,-0.101824,-0.222755,-2.001297,-2.101684,0.100387,0.668016,-0.000719,0.222630
309,El Salvador,married people having an affair,-0.857323,-1.623313,-1.390681,-0.232632,-0.751023,-3.226212,-2.361981,-0.864231,-0.615871,-0.548432,-0.683447
310,El Salvador,gambling,-0.748737,-1.526598,-1.592798,0.066199,0.455808,-2.093247,-2.056810,-0.036438,0.485905,0.014881,0.470857
